# 🤖 AI Agent Bricks: Intelligent F1 Applications
*Build AI-powered applications with Formula 1 insights*

---

## 🎯 Learning Objectives

By the end of this demo, you'll understand:
- ✅ **Databricks AI capabilities** and agent framework
- ✅ **Building intelligent applications** with F1 data
- ✅ **MLflow integration** for model management
- ✅ **Production AI deployment** patterns

---

## 🧠 What We'll Explore

**AI-Powered F1 Applications:**
```
🤖 AI Agent Capabilities:
├── 📊 Predictive Analytics (race outcome prediction)
├── 💬 Conversational AI (F1 chatbot)
├── 🔍 Intelligent Search (driver performance insights)
├── 📈 Automated Insights (performance anomaly detection)
└── 🎯 Recommendation Engine (optimal race strategies)
```

### 💡 AI Use Cases for F1:
- **Race Strategy Optimization** - AI-powered pit stop timing
- **Driver Performance Analysis** - ML models for consistency prediction
- **Fan Engagement** - Conversational AI for F1 questions
- **Predictive Maintenance** - Car performance monitoring
- **Broadcasting Intelligence** - Real-time race commentary insights

### 🎯 Implementation Patterns:
- **Model Training** with MLflow tracking
- **Feature Engineering** from F1 telemetry data
- **Real-time Inference** during race weekends
- **A/B Testing** for strategy optimization
- **Continuous Learning** from race outcomes

**Continue to the next notebook:** `07_SQL_Editor.sql`

**🏁 Ready to write powerful SQL queries? Let's analyze F1 data with SQL! 📊**

# 🤖 AI Agent Bricks: Build Intelligent Applications
*Create AI-powered F1 chatbots and intelligent apps in 3 minutes*

---

## 🎯 Learning Objectives

By the end of this guide, you'll understand:
- ✅ **AI Agents fundamentals** and key components
- ✅ **Vector Search integration** for knowledge retrieval
- ✅ **F1 Q&A chatbot** building blocks
- ✅ **Agent types and use cases** for different scenarios

---

## 🧠 What Are AI Agents?

**AI Agents** are intelligent applications that can understand natural language, access your data, and provide informed responses or take actions.

### 🔧 Key Components:

#### 1. **Foundation Models** 🤖
- **Large Language Models (LLMs)** for understanding and generation
- **Embeddings models** for semantic search and similarity
- **Databricks Model Serving** for scalable AI inference

#### 2. **Vector Search** 🔍
- **Semantic search** across your data
- **Similarity matching** for relevant information retrieval
- **Real-time indexing** of structured and unstructured data

#### 3. **Agent Framework** 🏗️
- **Tool calling** to access databases and APIs
- **Multi-turn conversations** with context memory
- **Response formatting** and safety controls

#### 4. **Agent Playground** 🎮
- **Interactive testing** environment
- **Conversation debugging** and refinement
- **Performance evaluation** tools

## 🏎️ F1 Q&A Bot Example

Let's design an intelligent F1 chatbot using your workshop data!

**[Screenshot: AI Agent interface showing F1 chatbot conversation with driver statistics queries]**
*📁 Image location: `images/06_f1_chatbot_demo.png`*
*Screenshot guidance: Show the Agent Playground with a conversation about F1 drivers, including questions like "Who has the most wins?" and the bot's responses with data*

### 🎯 F1 Bot Capabilities:

```
🏁 "Who is the most successful F1 driver of all time?"
Bot: "Based on our F1 database, Lewis Hamilton leads with 103 career wins 
     and 198 podium finishes across 310 races..."

🏎️ "Show me drivers from Britain with more than 20 wins"
Bot: "Here are British drivers with 20+ wins:
     • Lewis Hamilton: 103 wins
     • Nigel Mansell: 31 wins
     • Jackie Stewart: 27 wins..."

📊 "What's the trend in F1 safety over the decades?"
Bot: "F1 safety has dramatically improved. In the 1960s-70s, we saw frequent 
     DNFs due to mechanical failures. Modern F1 (2000+) shows much higher 
     completion rates and safety innovations..."
```

## 🏗️ Building Your F1 Agent: Step-by-Step

### Step 1: Prepare Your Data 📊

**[Screenshot: Data preparation interface showing F1 tables being indexed for vector search]**
*📁 Image location: `images/06_data_preparation.png`*
*Screenshot guidance: Show the process of selecting F1 tables (driver standings, race results) for inclusion in the agent's knowledge base*

#### Data Sources for F1 Agent:
```sql
-- Driver knowledge base
SELECT 
  full_name,
  nationality,
  total_career_points,
  wins,
  podiums,
  'Driver profile and career statistics' as content_type
FROM main.default.gold_driver_standings

-- Race insights  
SELECT
  season,
  total_races,
  unique_drivers,
  completion_rate,
  'Season statistics and trends' as content_type  
FROM main.default.gold_season_stats
```

#### Text Preparation:
- **Driver profiles:** "Lewis Hamilton is a British driver with 103 career wins..."
- **Race summaries:** "The 2023 F1 season featured 22 races with 20 unique drivers..."
- **Historical insights:** "F1's Hybrid Era (2014+) introduced new power units..."

In [ ]:
-- Step 2: Create Vector Search Index
-- This SQL creates the source table for our vector search index

-- First, let's create a consolidated table with all F1 knowledge
CREATE OR REPLACE TABLE main.default.f1_agent_knowledge AS

-- Driver knowledge
SELECT 
  d.full_name as entity_name,
  'driver' as entity_type,
  CONCAT(
    d.full_name, ' is a ', d.nationality, ' Formula 1 driver with ',
    d.wins, ' race wins, ', d.podiums, ' podium finishes, and ',
    d.total_career_points, ' career points across ', d.total_races, ' races.'
  ) as content_text,
  'Driver profile' as content_type,
  map(
    'nationality', d.nationality,
    'wins', CAST(d.wins AS STRING),
    'podiums', CAST(d.podiums AS STRING)
  ) as metadata
FROM main.default.gold_driver_standings d
WHERE d.wins > 0

UNION ALL

-- Season statistics
SELECT 
  CONCAT('Season ', s.season) as entity_name,
  'season' as entity_type,
  CONCAT(
    'The ', s.season, ' Formula 1 season had ', s.total_races, 
    ' races with ', s.unique_drivers, ' drivers competing. ',
    'The average completion rate was ', ROUND(s.completion_rate * 100, 1), '%.'
  ) as content_text,
  'Season statistics' as content_type,
  map(
    'season', CAST(s.season AS STRING),
    'races', CAST(s.total_races AS STRING)
  ) as metadata
FROM main.default.gold_season_stats s
WHERE s.season >= 2000

-- Now create the Vector Search index using the Databricks UI
-- 1. Go to Catalog Explorer
-- 2. Select the table main.default.f1_agent_knowledge
-- 3. Click "Create" → "Vector Search Index"
-- 4. Configure as follows:
--    - Index name: f1_knowledge_index
--    - Embedding model: databricks-bge-large-en
--    - Text column: content_text
--    - Primary key: entity_name
--    - Index type: delta_sync

In [ ]:
# Step 3: Configure Agent Framework with Python SDK

# Import the required libraries for AI agents
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
import os

# Initialize clients
ws = WorkspaceClient()
vs_client = VectorSearchClient(workspace_client=ws)

# Define our vector search endpoint for the agent
vs_index_fullname = "main.default.f1_knowledge_index"

# Define the system prompt for our F1 expert agent
f1_system_prompt = """
You are an expert Formula 1 analyst and historian with access to comprehensive 
F1 data from 1950-2023. You can answer questions about:

• Driver careers, statistics, and achievements
• Race results, season trends, and historical analysis  
• Team performance and constructor championships
• F1 regulations, eras, and technical evolution

Always provide specific data points and statistics when available. 
If you're unsure about something, clearly state your uncertainty.
Keep responses conversational but informative.
"""

# Function to perform vector search using our indexed knowledge
def search_f1_knowledge(query, limit=5):
    """Search the F1 knowledge base for relevant information"""
    results = vs_client.similarity_search(
        index_name=vs_index_fullname,
        query_text=query,
        num_results=limit
    )
    
    # Extract and return the content from search results
    retrieved_docs = []
    for match in results.matches:
        retrieved_docs.append({
            'content': match.document["content_text"],
            'entity': match.document["entity_name"],
            'type': match.document["entity_type"],
            'score': match.score
        })
    
    return retrieved_docs

# Example search query
print("Example vector search for: 'best British drivers'")
results = search_f1_knowledge("best British drivers")
for idx, doc in enumerate(results):
    print(f"\nResult {idx+1}: {doc['entity']} ({doc['score']:.2f})")
    print(f"Content: {doc['content']}")

### Step 4: Test in Playground 🎮

**[Screenshot: Agent Playground showing test conversation with F1 questions and responses]**
*📁 Image location: `images/06_playground_testing.png`*
*Screenshot guidance: Show active testing of the F1 agent with various questions and the agent's data-backed responses*

#### Test Questions:
- **Basic facts:** "How many races did Michael Schumacher win?"
- **Comparisons:** "Compare Lewis Hamilton and Ayrton Senna's careers"
- **Trends:** "How has F1 competitiveness changed over time?"
- **Complex queries:** "Which nationality has produced the most F1 champions?"

### Step 5: Deploy and Monitor 🚀

**[Screenshot: Agent deployment interface showing endpoint creation and monitoring dashboard]**
*📁 Image location: `images/06_agent_deployment.png`*
*Screenshot guidance: Show the agent deployment process with endpoint setup and monitoring metrics*

#### Deployment Options:
- **REST API endpoint** for application integration
- **Web interface** for direct user interaction
- **Slack/Teams bot** for team collaboration
- **Embedded widget** for dashboard integration

## 🎨 Agent Types and Use Cases

Databricks AI Agents can be configured for different use cases depending on your business needs:

| **Agent Type** | **Purpose** | **Example F1 Use Case** |
|---------------|-------------|------------------------|
| **SQL Agent** 📊 | Convert natural language to SQL | "Show top British drivers by wins" → SQL query |
| **RAG Agent** 🔍 | Retrieve knowledge & answer questions | Answer questions about F1 history and statistics |
| **Function Agent** 🛠️ | Execute actions using data | Compare driver performance across seasons |
| **Multi-Agent** 🤝 | Specialized agents for complex tasks | Complete race strategy analysis system |

In [ ]:
# SQL Agent example - natural language to SQL conversion
from databricks.sdk.service.catalog import TableFullName

# Define a function for our SQL Agent functionality
def f1_sql_agent(query):
    """Convert natural language to SQL for F1 data analysis"""
    print(f"Converting query: '{query}'")
    print("Tables available: gold_driver_standings, gold_race_results, gold_season_stats")
    
    # This would use the LLM in a real implementation
    # For this example, we'll simulate the conversion for demonstration
    
    example_queries = {
        "Show me British drivers with the most podiums": """
            SELECT full_name, nationality, podiums 
            FROM main.default.gold_driver_standings 
            WHERE nationality = 'British' 
            ORDER BY podiums DESC 
            LIMIT 10
        """,
        
        "Which drivers have won more than 20 races?": """
            SELECT full_name, nationality, wins
            FROM main.default.gold_driver_standings
            WHERE wins > 20
            ORDER BY wins DESC
        """
    }
    
    # Find the closest match for demo purposes
    closest_match = None
    for example in example_queries:
        if query.lower() in example.lower() or example.lower() in query.lower():
            closest_match = example
            break
    
    if closest_match:
        sql = example_queries[closest_match]
        print(f"\nGenerated SQL:\n{sql}")
        print("\nExecuting query...")
        
        # In a real agent, we would execute the SQL here
        print("Results would be displayed here")
        
        return sql
    else:
        print("No matching query template found for this example")
        return None

# Test our SQL Agent with sample queries
test_query = "Show me British drivers with the most podiums"
f1_sql_agent(test_query)
<VSCode.Cell id="#VSC-23060738" language="python">
# RAG (Retrieval-Augmented Generation) Agent Example
# This shows how to build a simple RAG agent for F1 knowledge

# Import necessary libraries
import json
from datetime import datetime

# 1. Define our retrieval function (using the search function we created earlier)
def retrieve_f1_knowledge(query, limit=3):
    """Retrieve relevant F1 knowledge for a given query"""
    # In a real implementation, this would call our vector search
    # For the example, we'll simulate the results
    
    print(f"Searching for: {query}")
    
    # Simulate search results
    if "hamilton" in query.lower():
        results = [
            {"content": "Lewis Hamilton is a British Formula 1 driver with 103 race wins, 198 podium finishes, and 4,639.5 career points across 310 races.", "entity": "Lewis Hamilton", "score": 0.92},
            {"content": "Lewis Hamilton has won 7 World Championships (2008, 2014, 2015, 2017, 2018, 2019, 2020), tied with Michael Schumacher for most all-time.", "entity": "Lewis Hamilton", "score": 0.87}
        ]
    elif "schumacher" in query.lower():
        results = [
            {"content": "Michael Schumacher is a German Formula 1 driver with 91 race wins, 155 podium finishes, and 1,566 career points across 307 races.", "entity": "Michael Schumacher", "score": 0.94},
            {"content": "Michael Schumacher won 7 World Championships (1994, 1995, 2000, 2001, 2002, 2003, 2004), tied with Lewis Hamilton for most all-time.", "entity": "Michael Schumacher", "score": 0.89}
        ]
    else:
        results = [
            {"content": "Formula 1 is the highest class of international racing for single-seater formula racing cars.", "entity": "Formula 1", "score": 0.75},
            {"content": "The first Formula 1 World Championship race was held in 1950 at Silverstone, United Kingdom.", "entity": "Formula 1 History", "score": 0.72}
        ]
        
    return results

# 2. Create the RAG agent function
def f1_rag_agent(query, conversation_history=[]):
    """
    A simple RAG agent that answers F1 questions
    
    Args:
        query: User's question about F1
        conversation_history: List of previous exchanges
    
    Returns:
        Agent's response based on retrieved knowledge
    """
    print(f"🤖 Processing query: {query}")
    
    # Step 1: Retrieve relevant knowledge
    relevant_docs = retrieve_f1_knowledge(query)
    
    # Step 2: Format retrieved knowledge for context
    context = "\n".join([f"- {doc['content']}" for doc in relevant_docs])
    print(f"\nRelevant knowledge found:\n{context}")
    
    # Step 3: Generate response (in a real agent, this would use an LLM)
    # For this example, we'll simulate a response
    if "hamilton" in query.lower() and "schumacher" in query.lower():
        response = """Based on our F1 database:

Lewis Hamilton and Michael Schumacher are tied with 7 World Championships each, making them the most successful F1 drivers by championship wins.

Hamilton has 103 race wins compared to Schumacher's 91, giving Hamilton the edge in total victories. Hamilton achieved his success across 310 races, while Schumacher competed in 307 races during his career.

Both drivers dominated their respective eras, with Schumacher winning 5 consecutive championships with Ferrari (2000-2004) and Hamilton winning 6 championships in 7 years with Mercedes (2014-2020)."""

    elif "hamilton" in query.lower():
        response = """Based on our F1 database:

Lewis Hamilton is a British Formula 1 driver who has won 103 races and achieved 198 podium finishes throughout his career. He has competed in 310 races and accumulated 4,639.5 career points.

Hamilton has won 7 World Championships (2008, 2014, 2015, 2017, 2018, 2019, 2020), tied with Michael Schumacher for the most championships in F1 history.

He began his F1 career with McLaren in 2007 before moving to Mercedes in 2013, where he has achieved most of his success during the turbo-hybrid era."""

    elif "schumacher" in query.lower():
        response = """Based on our F1 database:

Michael Schumacher is a German Formula 1 driver who won 91 races and achieved 155 podium finishes during his career. He competed in 307 races across two career phases (1991-2006 and 2010-2012).

Schumacher won 7 World Championships (1994, 1995, 2000, 2001, 2002, 2003, 2004), with his most dominant period being at Ferrari where he won 5 consecutive titles.

He is known for his exceptional race craft, wet weather driving, and his role in transforming Ferrari into a dominant team in the early 2000s."""

    else:
        response = """I'd be happy to answer your F1 question, but I need more specific information. You can ask about:

- Driver statistics and comparisons
- Team performance and history
- Season results and championships
- F1 regulations and technical aspects

For example, try asking "Who has won the most F1 championships?" or "Tell me about Ferrari's history in F1."
"""
    
    # Add this exchange to conversation history
    conversation_history.append({
        "timestamp": datetime.now().isoformat(),
        "query": query,
        "response": response
    })
    
    return response

# Test the RAG agent
print("==== F1 RAG Agent Demo ====")
conversation = []

# First query
query1 = "Tell me about Lewis Hamilton"
response1 = f1_rag_agent(query1, conversation)
print(f"\n👤 User: {query1}")
print(f"\n🤖 Agent: {response1}\n")

# Follow-up query
query2 = "How does he compare to Schumacher?"
response2 = f1_rag_agent(query2, conversation)
print(f"\n👤 User: {query2}")
print(f"\n🤖 Agent: {response2}")
<VSCode.Cell id="#VSC-58fc5588" language="markdown">
## 📊 Monitoring and Optimization

### Agent Performance Metrics 📈

**[Screenshot: Agent analytics dashboard showing usage patterns, response times, and user satisfaction]**
*📁 Image location: `images/06_agent_analytics.png`*
*Screenshot guidance: Show metrics dashboard with conversation volume, response accuracy, user ratings, and performance trends*

#### Key Metrics:
- **Response accuracy** (user feedback scores)
- **Query resolution rate** (successful vs. failed queries)
- **Response latency** (time to first response)
- **User engagement** (conversation length, return users)
- **Cost optimization** (token usage, model calls)

### Continuous Improvement 🔄
- **A/B testing** different system prompts
- **Fine-tuning** on domain-specific data
- **Knowledge base updates** with new F1 data
- **User feedback integration** for response quality

## 💡 Best Practices for F1 AI Agents

| **Category** | **Key Practices** | **Why It Matters** |
|--------------|-------------------|-------------------|
| **Data Preparation** 📋 | • Clean structured data<br>• Include context in text chunks<br>• Regular updates with latest results | High-quality data leads to accurate, relevant responses |
| **Prompt Engineering** 🎯 | • Domain-specific system prompts<br>• Clear data citation instructions<br>• Consistent output formatting | Well-crafted prompts significantly improve agent performance |
| **Architecture** 🏗️ | • Split complex tasks into steps<br>• Use appropriate agent type for use case<br>• Design clear tool interfaces | The right architecture ensures efficient, maintainable agents |
| **Testing & Monitoring** 📊 | • Test with diverse queries<br>• Monitor accuracy metrics<br>• Collect user feedback | Continuous improvement requires visibility into performance |

### Example System Prompt for F1 Agent:

```
You are an expert Formula 1 analyst with access to comprehensive F1 data.
When answering questions:
1. Always cite specific statistics when available
2. Format driver names in bold
3. Present numerical comparisons in tables when comparing multiple drivers
4. If you're uncertain, clearly state the limitations of your knowledge
5. When discussing historical trends, consider regulation changes and technological evolution
```
<VSCode.Cell id="#VSC-dface9dc" language="markdown">
## ✅ AI Agents Complete!

**🎉 Excellent! You've learned the fundamentals of building intelligent F1 applications!**

### What You've Accomplished:
- ✅ **Understood AI Agent architecture** and key components
- ✅ **Designed F1 chatbot** with comprehensive capabilities
- ✅ **Learned agent types** (SQL, RAG, Function Calling, Multi-Agent)
- ✅ **Explored advanced features** (memory, safety, monitoring)
- ✅ **Applied best practices** for domain-specific agents

### 🏗️ Your F1 Agent Architecture:
```
🏎️ F1 Data Sources (Gold Tables)
    ↓
🔍 Vector Search Index (Semantic retrieval)
    ↓  
🤖 AI Agent Framework (LLM + Tools)
    ↓
🎮 Interactive Interface (Chat/API)
    ↓
📊 Analytics & Monitoring
```

### 🎯 Agent Capabilities Built:
- **Driver statistics** and career analysis
- **Historical insights** and trend analysis
- **Comparative analysis** between drivers/eras
- **Natural language** data exploration

## 🚀 Next Steps

Ready to explore SQL analytics and visualization?

### Immediate Actions:
1. **🤖 Plan your F1 agent:** Define specific use cases and user personas
2. **📊 Prepare data sources:** Use gold tables from notebook 02
3. **🔍 Create vector index:** Start with driver profiles and race summaries

### Next Notebook:
**➡️ [07_SQL_Editor.sql](07_SQL_Editor.sql)**
- Build analytical queries for F1 insights
- Create interactive visualizations
- Design executive dashboards

### Advanced Exploration:
- **🎮 Agent Playground:** Test different conversation flows
- **🔧 Custom functions:** Build F1-specific tools and integrations
- **📈 Multi-modal agents:** Integrate race footage and telemetry data

### 💡 Pro Tips:
- **🎯 Start simple** with basic Q&A before advanced features
- **📊 Use structured data** from your gold tables for reliable responses
- **🔄 Iterate based on user feedback** and conversation patterns
- **🛡️ Implement safety controls** for production deployment

**🤖 Ready to build the future of F1 analytics with AI! 🏎️**